In [1]:
import torch

import pandas as pd
import numpy as np

In [2]:
x = torch.rand(5,3)
print(x)
y = torch.ones(5,3)
print(y)

tensor([[0.4640, 0.1705, 0.0710],
        [0.3133, 0.5607, 0.0540],
        [0.3792, 0.6071, 0.6906],
        [0.9480, 0.2464, 0.3750],
        [0.5173, 0.2017, 0.6586]])
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])


In [3]:
z = x + y
print(z)

tensor([[1.4640, 1.1705, 1.0710],
        [1.3133, 1.5607, 1.0540],
        [1.3792, 1.6071, 1.6906],
        [1.9480, 1.2464, 1.3750],
        [1.5173, 1.2017, 1.6586]])


# Advantages of Pytorch over Numpy
## Automatic Differentiation
## Running on GPUs

In [5]:
df = pd.read_csv('tictactoemoves.csv')

In [6]:
print(df.head())

   C1  C2  C3  C4  C5  C6  C7  C8  C9  player  move
0   1   1   1   1   0   0   1   0   0      -1    -1
1   1   1   1   1   0   0   0   1   0      -1    -1
2   1   1   1   1   0   0   0   0   1      -1    -1
3   1   1   1   1   0   0   0  -1  -1      -1    -1
4   1   1   1   1   0   0  -1   0  -1      -1    -1


In [7]:
X = df[['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'player']].values
y = df['move'].values

In [8]:
print(y)

[-1 -1 -1 ...  3  5  1]


In [16]:
y_encoded = np.zeros([len(y), 10])

for i in range(0, len(y)):
    if(y[i] == -1):
        y_encoded[i][0] = 1.0
    else:
        y_encoded[i][int(y[i])] = 1.0

In [18]:
print(y_encoded[-2])

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [20]:
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y_encoded, dtype=torch.float32)

In [22]:
print(y_tensor)

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.]])


### train,test sets

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
#X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.1, random_state=42)

In [27]:
print(len(X_test))

548


## NN Model

In [32]:
import torch.nn as nn

In [109]:
class TicTacToeTrainer(nn.Module):

    def __init__(self):
        super(TicTacToeTrainer, self).__init__()

        self.fc1 = nn.Linear(10, 32)
        self.fc2 = nn.Linear(32, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 10) #logits as output

        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)

        return x
       

In [111]:
model = TicTacToeTrainer()

In [113]:
x_eval = torch.tensor([[0,-1,-1,-1,-1,0,1,1,-1,1]],dtype=torch.float32)
output = model(x_eval)


In [74]:
print(output)

tensor([[-0.0228,  0.0253,  0.0159,  0.0110, -0.2053,  0.2764, -0.1847, -0.0421,
          0.1159,  0.2017]], grad_fn=<AddmmBackward0>)


# Training

## Softmax + Cross-Entropy

In [115]:
criterion = nn.CrossEntropyLoss()

## Optimizer

In [118]:
import torch.optim as optim

In [120]:
optimizer =  optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

## Training

In [122]:
X_test = X_tensor
X_train = X_tensor

y_test = y_tensor
y_train = y_tensor

In [128]:
num_epochs = 1000 # 1 epoch, 1 traversal through the training set
model.train()
for epoch in range(num_epochs):
    optimizer.zero_grad() #zero out the gradients
    outputs = model(X_train) #run the forward pass
    loss = criterion(outputs, y_train) # compute the loss
    loss.backward() #gradient calculation happens from here
    optimizer.step() #takes the step based on the gradients and lr

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.6f}')

Epoch [10/1000], Loss: 0.016147
Epoch [20/1000], Loss: 0.016129
Epoch [30/1000], Loss: 0.016113
Epoch [40/1000], Loss: 0.016097
Epoch [50/1000], Loss: 0.016076
Epoch [60/1000], Loss: 0.016059
Epoch [70/1000], Loss: 0.016044
Epoch [80/1000], Loss: 0.016025
Epoch [90/1000], Loss: 0.016006
Epoch [100/1000], Loss: 0.015987
Epoch [110/1000], Loss: 0.015972
Epoch [120/1000], Loss: 0.015952
Epoch [130/1000], Loss: 0.015935
Epoch [140/1000], Loss: 0.015916
Epoch [150/1000], Loss: 0.015896
Epoch [160/1000], Loss: 0.015878
Epoch [170/1000], Loss: 0.015859
Epoch [180/1000], Loss: 0.015842
Epoch [190/1000], Loss: 0.015822
Epoch [200/1000], Loss: 0.015805
Epoch [210/1000], Loss: 0.015785
Epoch [220/1000], Loss: 0.015767
Epoch [230/1000], Loss: 0.015749
Epoch [240/1000], Loss: 0.015730
Epoch [250/1000], Loss: 0.015712
Epoch [260/1000], Loss: 0.015691
Epoch [270/1000], Loss: 0.015675
Epoch [280/1000], Loss: 0.015654
Epoch [290/1000], Loss: 0.015637
Epoch [300/1000], Loss: 0.015616
Epoch [310/1000], L

In [130]:
# Evaluating
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs.data, 1)
    _, actual = torch.max(y_test.data, 1)

    accuracy = (predicted == actual).sum().item()/actual.size(0)*100.0

    print(f'Test Accuracy : {accuracy}')

Test Accuracy : 99.96349032493612


In [97]:
# Evaluating
model.eval()
with torch.no_grad():
    outputs = model(X_train)
    _, predicted = torch.max(outputs.data, 1)
    _, actual = torch.max(y_train.data, 1)

    accuracy = (predicted == actual).sum().item()/actual.size(0)*100.0

    print(f'Training Accuracy : {accuracy}')

Training Accuracy : 86.59229208924948


In [103]:
invalid = 0
invalid_states = []
for i in range(0, len(predicted)):
    isEmpty = False 
    if(actual[i] != predicted[i]):
        isEmpty = False 
        if(X_test[i,predicted[i]-1] == -1):
            isEmpty = True
            
        else:
            invalid = invalid + 1
            invalid_states.append({X_test[i].to(torch.int32),actual[i],predicted[i]})
        print(X_test[i].to(torch.int32),actual[i],predicted[i], isEmpty)


tensor([ 0, -1, -1, -1,  1,  0,  1, -1, -1,  1], dtype=torch.int32) tensor(2) tensor(3) True
tensor([ 0, -1,  1,  1,  1,  0, -1,  0, -1,  1], dtype=torch.int32) tensor(7) tensor(2) True
tensor([-1,  0,  1, -1, -1,  0, -1,  1,  1,  0], dtype=torch.int32) tensor(7) tensor(1) True
tensor([-1,  0,  1,  0, -1,  1, -1,  1,  0,  1], dtype=torch.int32) tensor(1) tensor(5) True
tensor([-1, -1,  1, -1,  0, -1, -1,  0,  1,  1], dtype=torch.int32) tensor(2) tensor(6) True
tensor([-1, -1,  0, -1,  0,  1,  1,  1, -1,  0], dtype=torch.int32) tensor(9) tensor(2) True
tensor([ 0, -1,  1, -1, -1, -1,  0,  1, -1,  1], dtype=torch.int32) tensor(4) tensor(2) True
tensor([ 0,  1,  1, -1, -1, -1,  0, -1,  1,  0], dtype=torch.int32) tensor(4) tensor(6) True
tensor([-1, -1,  0, -1,  1,  1, -1, -1, -1,  0], dtype=torch.int32) tensor(4) tensor(1) True
tensor([-1, -1,  1, -1, -1, -1, -1,  1,  0,  0], dtype=torch.int32) tensor(2) tensor(5) True
tensor([ 0, -1, -1,  1,  1,  0,  1,  0,  1,  0], dtype=torch.int32) te

In [105]:
torch.save(model.state_dict(), "TicTacToe_OverfittedWeights.pth")